# Privacy Meter Demo

## Setting up the Colab environment

If you are running it offline, you can skip to "Importing"

In [ ]:
# Clone the github repo
!git clone https://github.com/privacytrustlab/ml_privacy_meter.git

# Update the Colab environment
!pip install datasets==2.21.0 transformers==4.44.2 torch==2.4.1 torchvision==0.19.1 torchaudio

In [ ]:
# Change the directory to the cloned repo
import sys
sys.path.append('/content/ml_privacy_meter')

%cd ml_privacy_meter

/content/ml_privacy_meter


## Importing

In [2]:
import math
import time

import numpy as np
import torch
import yaml
from torch.utils.data import Subset

from audit import get_average_audit_results, audit_models, sample_auditing_dataset
from get_signals import get_model_signals
from models.utils import load_models, train_models, split_dataset_for_training
from util import (
    check_configs,
    setup_log,
    initialize_seeds,
    create_directories,
    load_dataset,
)

# Enable benchmark mode in cudnn to improve performance when input sizes are consistent
torch.backends.cudnn.benchmark = True

/home/jiashu/miniconda/envs/privacy_meter/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load config

In [4]:
configs = "configs/config.yaml"
with open(configs, "rb") as f:
        configs = yaml.load(f, Loader=yaml.Loader)

# Validate configurations
check_configs(configs)

## Setting up

In [5]:
# Validate configurations
check_configs(configs)

# Initialize seeds for reproducibility
initialize_seeds(configs["run"]["random_seed"])

# Create necessary directories
log_dir = configs["run"]["log_dir"]
directories = {
    "log_dir": log_dir,
    "report_dir": f"{log_dir}/report",
    "signal_dir": f"{log_dir}/signals",
    "data_dir": configs["data"]["data_dir"],
}
create_directories(directories)

# Set up logger
logger = setup_log(
    directories["report_dir"], "time_analysis", configs["run"]["time_log"]
)

start_time = time.time()

## Load dataset

In [6]:
baseline_time = time.time()
dataset, population = load_dataset(configs, directories["data_dir"], logger)
logger.info("Loading dataset took %0.5f seconds", time.time() - baseline_time)

2024-10-25 22:24:56,396 INFO     Load data from data/cifar10.pkl
2024-10-25 22:24:56,397 INFO     The whole dataset size: 60000
2024-10-25 22:24:56,398 INFO     Loading dataset took 0.09912 seconds


## Load or train models

In [8]:
# Define experiment parameters
num_experiments = configs["run"]["num_experiments"]
num_reference_models = configs["audit"]["num_ref_models"]
num_model_pairs = max(math.ceil(num_experiments / 2.0), num_reference_models + 1)

# Load or train models
baseline_time = time.time()
models_list, memberships = load_models(
    log_dir, dataset, num_model_pairs * 2, configs, logger
)
if models_list is None:
    # Split dataset for training two models per pair
    data_splits, memberships = split_dataset_for_training(
        len(dataset), num_model_pairs
    )
    models_list = train_models(
        log_dir, dataset, data_splits, memberships, configs, logger
    )
logger.info(
    "Model loading/training took %0.1f seconds", time.time() - baseline_time
)


2024-10-25 22:26:25,183 INFO     Loading model 0
2024-10-25 22:26:25,202 INFO     Loading model 1
2024-10-25 22:26:25,217 INFO     Loading model 2
2024-10-25 22:26:25,230 INFO     Loading model 3
2024-10-25 22:26:25,243 INFO     Model loading/training took 0.1 seconds


## Prepare auditing dataset

In [9]:
auditing_dataset, auditing_membership = sample_auditing_dataset(
        configs, dataset, logger, memberships
    )

# Also downsample the population set size if specified in the config
population = Subset(
    population,
    np.random.choice(
        len(population),
        configs["audit"].get("population_size", len(population)),
        replace=False,
    ),
)

## Compute signals

In [10]:
baseline_time = time.time()
signals = get_model_signals(models_list, auditing_dataset, configs, logger)
population_signals = get_model_signals(
        models_list, population, configs, logger, is_population=True
    )
logger.info("Preparing signals took %0.5f seconds", time.time() - baseline_time)

2024-10-25 22:27:11,146 INFO     Signals loaded from disk.
2024-10-25 22:27:11,147 INFO     Preparing signals took 0.00190 seconds


## Audit

In [11]:
# Perform the privacy audit
baseline_time = time.time()
target_model_indices = list(range(num_experiments))
mia_score_list, membership_list = audit_models(
        f"{directories['report_dir']}/exp",
        target_model_indices,
        signals,
        population_signals,
        auditing_membership,
        num_reference_models,
        logger,
        configs,
    )

if len(target_model_indices) > 1:
    logger.info(
        "Auditing privacy risk took %0.1f seconds", time.time() - baseline_time
    )

# Get average audit results across all experiments
if len(target_model_indices) > 1:
    get_average_audit_results(
        directories["report_dir"], mia_score_list, membership_list, logger
    )

logger.info("Total runtime: %0.5f seconds", time.time() - start_time)

2024-10-25 22:27:45,346 INFO     Fine-tuning offline_a using paired model 1
2024-10-25 22:27:45,361 INFO     offline_a=0.00: AUC 0.7935
2024-10-25 22:27:45,373 INFO     offline_a=0.10: AUC 0.7893
2024-10-25 22:27:45,385 INFO     offline_a=0.20: AUC 0.7851
2024-10-25 22:27:45,398 INFO     offline_a=0.30: AUC 0.7803
2024-10-25 22:27:45,410 INFO     offline_a=0.40: AUC 0.7755
2024-10-25 22:27:45,422 INFO     offline_a=0.50: AUC 0.7702
2024-10-25 22:27:45,433 INFO     offline_a=0.60: AUC 0.7643
2024-10-25 22:27:45,445 INFO     offline_a=0.70: AUC 0.7576
2024-10-25 22:27:45,467 INFO     offline_a=0.80: AUC 0.7483
2024-10-25 22:27:45,479 INFO     offline_a=0.90: AUC 0.7352
2024-10-25 22:27:45,491 INFO     offline_a=1.00: AUC 0.6820
2024-10-25 22:27:45,492 INFO     The best offline_a is 0.0
2024-10-25 22:27:45,505 INFO     Target Model 0: AUC 0.7889, TPR@0.1%FPR of 0.0104, TPR@0.0%FPR of 0.0003
2024-10-25 22:27:49,066 INFO     Auditing the privacy risks of target model 0 costs 3.7 seconds
202

<Figure size 640x480 with 0 Axes>